In [1]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import HoverTool, CustomJS
from bokeh.models import ColumnDataSource, LinearColorMapper, ColorBar, FixedTicker
from bokeh.transform import linear_cmap
from bokeh.palettes import Category20, Viridis256, Inferno256, inferno, viridis, linear_palette, Turbo256, Spectral
from bokeh.transform import factor_cmap, factor_mark, transform
import pandas as pd
import numpy as np
output_notebook()

south=pd.read_csv('MANGOs_full_sample.csv')

Loading BokehJS ...

In [2]:
colors = LinearColorMapper(palette = list(Inferno256)[::-1],
                          high = max(south['Jmag']),
                          low = min(south['Jmag']))

color_array = ['#94D7F0' for i in range(len(south['TIC ID']))]
tics = [i for i in south['TIC ID']]
source = ColumnDataSource(
    data=dict(
        y=np.array(south['Planet Radius (R_Earth)']),
        x=np.array(south['Planet Eq Temp (K)']),
        z=np.array(south['Stellar Radius (R_Sun)'])*50,
        rs=np.array(south['Stellar Radius (R_Sun)']),
        j=np.array(south['Jmag']),
        p=np.array(south['Period (days)']),
        desc=np.array([str(i) for i in south['TOI']]),
        tics = tics,
                ))



w = figure(plot_width=800, plot_height=800, tools=['box_zoom', 'reset'], title="Hover to check target info", 
           x_axis_type = 'log', y_axis_type = 'log', x_axis_label = 'Equilibrium Temperature (K)', 
           y_axis_label = 'Radius (Earth rad)',)
w.scatter('x', 'y', size='z', source=source, marker='circle', fill_color=transform('j', colors), 
          line_color = 'black')

code = ''' 
for (let i in cb_data.renderer.data_source.data['color'])
    cb_data.renderer.data_source.data['color'][i] = colors[i];

if (cb_data.index.indices != null) {
    const hovered_index = cb_data.index.indices[0];
    const hovered_ipc_class = cb_data.renderer.data_source.data['tics'][hovered_index];
    for (let i = 0; i < cb_data.renderer.data_source.data['tics'].length; i++) {
        if (cb_data.renderer.data_source.data['tics'][i] == hovered_ipc_class)
            cb_data.renderer.data_source.data['color'][i] = 'pink';
    }
}
cb_data.renderer.data_source.change.emit();
'''
tooltips=[
            ("Radius", "@y"),
            ("Teq", "@x"),
            ("TOI", "@desc"),
            ("Jmag", "@j"),
            ("Period", "@p"),
            ("Rs", "@rs")
                ]

callback = CustomJS(args = dict(tics = tics, colors = color_array), code = code)
hover = HoverTool(tooltips = tooltips,  callback = callback)
w.add_tools(hover)

color_bar = ColorBar(color_mapper = colors,
                     location = (0,0),
                     title = 'Jmag',
                    )
 
w.add_layout(color_bar, 'right')

show(w)